In [7]:
import arcpy, os, arcgis, zipfile, sys, glob
from arcgis import GIS
from arcpy import env

inFeature = r'C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime\DC Crime.gdb\Major_Sports_Venues'
clipFeature = r'C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime\DC Crime.gdb\DistrictBoundary'
buffDist = "250 Meters"
localDir = r'C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime'
portal = 'https://esrinstallations.maps.arcgis.com'
user = "james_jones_installations"
pw = "QWerty654321@!"
outName = "DC_Sports_Venues"

In [8]:
if os.path.exists(r'C:\temp') == False:
    os.mkdir(r'C:\temp')

In [9]:
tempWS = r"C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime\DC Crime.gdb"
stageFolder = r'C:\temp'
tempClip = os.path.join(tempWS, "tempClip")
tempBuff = os.path.join(tempWS, "tempBuff")
tempShp = os.path.join(stageFolder, "tempShp.shp")
outZip = ''

In [11]:
env.overwriteOutput = 1
arcpy.Clip_analysis(inFeature, clipFeature, tempClip)

<Result 'C:\\Users\\jame9353\\Documents\\ArcGIS\\Projects\\DC Crime\\DC Crime.gdb\\tempClip'>

In [12]:
arcpy.Buffer_analysis(tempClip, tempBuff, buffDist)

<Result 'C:\\Users\\jame9353\\Documents\\ArcGIS\\Projects\\DC Crime\\DC Crime.gdb\\tempBuff'>

In [13]:
arcpy.CopyFeatures_management(tempBuff, tempShp)

<Result 'C:\\temp\\tempShp.shp'>

In [14]:
def zipShapefilesInDir(inDir, outDir):   
    if not os.path.exists(inDir):  
        print("Input directory %s does not exist!" % inDir)
        return False  
    
    if not os.path.exists(outDir):   
        print ("Creating output directory %s" %outDir)  
        os.mkdir(outDir)  
  
    print("Zipping shapefile(s) in folder %s to output folder %s" % (inDir, outDir))  
   
    for inShp in glob.glob(os.path.join(inDir, "*.shp")):
        global outZip
        outZip = os.path.join(outDir, os.path.splitext(os.path.basename(inShp))[0] + ".zip")  
   
        zipShapefile(inShp, outZip)  
    return True 

def zipShapefile(inShapefile, newZipFN):  
    print('Starting to Zip '+(inShapefile)+' to '+(newZipFN))
    
    if not (os.path.exists(inShapefile)):  
        print(inShapefile + ' Does Not Exist')
        return False  
   
    if (os.path.exists(newZipFN)):  
        print('Deleting '+newZipFN)  
        os.remove(newZipFN)  
    
    if (os.path.exists(newZipFN)):  
        print('Unable to Delete'+newZipFN) 
        return False  
  
    zipobj = zipfile.ZipFile(newZipFN,'w')  
   
    for infile in glob.glob( inShapefile.lower().replace(".shp",".*")):  
        if os.path.splitext(infile)[1].lower() != ".zip":  
            print("Zipping %s" % (infile))  
            zipobj.write(infile,os.path.basename(infile),zipfile.ZIP_DEFLATED)  
    
    zipobj.close()  
    return True 

In [15]:
zipShapefilesInDir(stageFolder, localDir)

Zipping shapefile(s) in folder C:\temp to output folder C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime
Starting to Zip C:\temp\tempShp.shp to C:\Users\jame9353\Documents\ArcGIS\Projects\DC Crime\tempShp.zip
Zipping c:\temp\tempShp.cpg
Zipping c:\temp\tempShp.dbf
Zipping c:\temp\tempShp.prj
Zipping c:\temp\tempShp.sbn
Zipping c:\temp\tempShp.sbx
Zipping c:\temp\tempShp.shp
Zipping c:\temp\tempShp.shp.xml
Zipping c:\temp\tempShp.shx


True

In [16]:
gis = GIS(url=portal, username=user, password=pw)

In [17]:
tempItem = gis.content.add({"title":outName}, outZip)

In [18]:
tempLyr= tempItem.publish(None)